In [1]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
from statsmodels.api import OLS
import sklearn.model_selection as skm
import sklearn.linear_model as skl
from sklearn.preprocessing import StandardScaler
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from functools import partial
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from ISLP.models import \
     (Stepwise,
      sklearn_selected,
      sklearn_selection_path)
from l0bnb import fit_path

In [14]:
Hitters = load_data('Hitters')
print(np.isnan(Hitters['Salary']).sum())
Hitters = Hitters.dropna()
print(Hitters.shape)
print(Hitters.dtypes)

59
(263, 20)
AtBat           int64
Hits            int64
HmRun           int64
Runs            int64
RBI             int64
Walks           int64
Years           int64
CAtBat          int64
CHits           int64
CHmRun          int64
CRuns           int64
CRBI            int64
CWalks          int64
League       category
Division     category
PutOuts         int64
Assists         int64
Errors          int64
Salary        float64
NewLeague    category
dtype: object


sklearnd doesn't have Cp (as I correctly saw in the titanic dataset feature selection), so here's a function to compute it by oneself.

In [8]:
def nCp(sigma2:float,     # variance
        estimator,
        X:pd.DataFrame,   # all data without the response
        Y:pd.Series) -> float:
    'Negative Cp statistic'
    n, p = X.shape
    Yhat = estimator.predict(X)
    RSS = np.sum((Y - Yhat)**2)
    return -(RSS + 2 * p * sigma2) / n

Estimating sigma2 (the variance) using the biggest model (= has the highest variance)

In [9]:
design = MS(Hitters.columns.drop('Salary')).fit(Hitters)
Y = np.array(Hitters['Salary'])
X = design.transform(Hitters)
sigma2 = OLS(Y,X).fit().scale

We want to use sigma2 as a const in the function nCp, so we freeze the function with that. This is then our "scorer", although I don't quite know what that means.

In [10]:
neg_Cp = partial(nCp, sigma2)

Apart from the scorer, we need a Search strategy. From the book Lab: The method `Stepwise.first_peak()` runs forward stepwise until any further additions to the model do not result in an improvement in the evaluation score. Similarly, the method `Stepwise.fixed_steps()` runs a fixed number of steps of stepwise search.

In [15]:
strategy = Stepwise.first_peak(design,                      # our model (with the intercept)
                               direction='forward',         # forward, backward or both
                               max_terms=len(design.terms)) # number of predictors

Now to implement forward selection. This is done with the function sklearn_selected() from the ISLP package. It takes a model from statsmodels and a search strategy, and then you can compute by using its fit() method.

In [ ]:
hitters_MSE = sklearn_selected(OLS, strategy)
hitters_MSE.fit(Hitters, # the response column is still in the dataset!
                Y)
print(hitters_MSE.selected_state_)
print(len(hitters_MSE.selected_state_))
print(dir(hitters_MSE))
print(hitters_MSE.__class__)
print(type(hitters_MSE)) # its the same as the line before

('Assists', 'AtBat', 'CAtBat', 'CHits', 'CHmRun', 'CRBI', 'CRuns', 'CWalks', 'Division', 'Errors', 'Hits', 'HmRun', 'League', 'NewLeague', 'PutOuts', 'RBI', 'Runs', 'Walks', 'Years')
19
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__sklearn_clone__', '__str__', '__subclasshook__', '__weakref__', '_build_request_for_signature', '_check_feature_names', '_check_n_features', '_estimator_type', '_get_default_requests', '_get_metadata_request', '_get_param_names', '_get_tags', '_more_tags', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_validate_data', '_validate_params', 'cv', 'fit', 'get_metadata_routing', 'get_params', 'model_', 'model_args', 'model_type', 'predict', 'results_', 'score', 'scoring

Now to use neg_Cp as the scoring argument for sklearn_selected(), so it doesn't use the MSE (and thus doesn't use the MSE which always results in the largest model)

In [21]:
hitters_Cp = sklearn_selected(OLS,
                              strategy,
                              scoring=neg_Cp)
hitters_Cp.fit(Hitters, Y)
print(hitters_Cp.selected_state_)
print(len(hitters_Cp.selected_state_))

('Assists', 'AtBat', 'CAtBat', 'CRBI', 'CRuns', 'CWalks', 'Division', 'Hits', 'PutOuts', 'Walks')
10


# Using CV and Validation Set Approach for choosing a model

Now we use fixed_steps instead of first_peak. I don't know exactly what the difference is, but I think when using MSE as training error for first_peak, it's the same as the fixed_steps for the length of the variables, since it will always add another variable, because the MSE will always be lower.